In [3]:
# RODAR ESSA CÉLULA PARA FAZER OS IMPORTS E DEFINIR AS FUNÇÕES
from scipy.interpolate import lagrange
import numpy as np
from sympy import Symbol
import sympy as sp

def interp_coeffs(xvals, yvals):
    """
    Return the divided-difference interpolating polynomial
    coefficients based on xvals and yvals. The degree of
    the interpolating polynomial will be inferred from data,
    which will be 1 less than the number of data points.
    """
    assert len(xvals)==len(yvals)
    nbr_data_points = len(xvals)

    # sort inputs by xvals =>
    data = sorted(zip(xvals, yvals), reverse=False)
    xvals, yvals = zip(*data)

    depth      = 1
    coeffs     = [yvals[0]]
    iter_yvals = yvals

    while depth < nbr_data_points:

        iterdata = []

        for i in range(len(iter_yvals)-1):

            delta_y = iter_yvals[i+1]-iter_yvals[i]
            delta_x = xvals[i+depth]-xvals[i]
            iterval = (delta_y/delta_x)
            iterdata.append(iterval)

            # append top-most entries from table to coeffs =>
            if i==0: coeffs.append(iterval)

        iter_yvals = iterdata
        depth+=1

    return(coeffs)




def interp_val(xvals, yvals):
    """
    Return a function representing the interpolating
    polynomial determined by xvals and yvals.
    """
    assert len(xvals)==len(yvals)
    nbr_data_points = len(xvals)

    # sort inputs by xvals =>
    data = sorted(zip(xvals, yvals), reverse=False)
    xvals, yvals = zip(*data)

    depth      = 1
    coeffs     = [yvals[0]]
    iter_yvals = yvals

    while depth < nbr_data_points:

        iterdata = []

        for i in range(len(iter_yvals)-1):

            delta_y = iter_yvals[i+1]-iter_yvals[i]
            delta_x = xvals[i+depth]-xvals[i]
            iterval = (delta_y/delta_x)
            iterdata.append(iterval)

            # append top-most entries in tree to coeffs =>
            if i==0: coeffs.append(iterval)

        iter_yvals = iterdata
        depth+=1

    def f(i):
        """
        Evaluate interpolated estimate at x.
        """
        terms  = []
        retval = 0

        for j in range(len(coeffs)):

            iterval   = coeffs[j]
            iterxvals = xvals[:j]
            for k in iterxvals: iterval*=(i-k)
            terms.append(iterval)
            retval+=iterval
        return(retval)
    return(f)

In [4]:
#################### LAGRANGE ############################
# SE FOR UM CASO DISCRETO, ONDE VC NAO TEM A FUNÇÃO, MUDAR OS VETORES X E Y A SEGUIR

X = Symbol('x')
x = np.array([0, 1, 2, 3])
y = np.array([1, 1, 7, 25])
L = {}
for i in range(len(x)):
    L[f'L{i}'] = 1
    for xi in x:
        if xi != x[i]:
            L[f'L{i}'] *= (X - xi)/(x[i] - xi)

for key in L:
    print(f'{key}: {sp.expand(L[key])}')
# IMPRIME TODOS OS POLINÔMIOS DE lAGRANGE

L0: -x**3/6 + x**2 - 11*x/6 + 1
L1: x**3/2 - 5*x**2/2 + 3*x
L2: -x**3/2 + 2*x**2 - 3*x/2
L3: x**3/6 - x**2/2 + x/3


In [8]:
# SE PRECISAR TESTAR ALGUM PONTO NOS POLINÔMIOS DADOS ACIMA, MUDAR pt E RODAR
pt = 1.5
for key in L:
    print(f'{key}({pt}) = {L[key].subs(X, pt)}')

L0(1.5) = -0.0625000000000000
L1(1.5) = 0.562500000000000
L2(1.5) = 0.562500000000000
L3(1.5) = -0.0625000000000000


In [9]:
# RETORNA OS COEFICIENTES FINAIS POLINÔMIO FINAL DE LAGRANGE COMPLETO 
poly = lagrange(x, y)
poly

poly1d([ 1.00000000e+00,  1.77635684e-15, -1.00000000e+00,  1.00000000e+00])

In [14]:
############################# NEWTON #############################
# SE FOR CASO DISCRETO, MUDAR VETORES X E Y
x = np.array([-2, -1, 0, 2])
y = (3/2)**x
divs = interp_coeffs(x, y)
divs
# RETORNA OS COEFICIENTES QUE SERÃO UTILIZADOS NO POLINÔMIO FINAL

[0.4444444444444444,
 0.2222222222222222,
 0.05555555555555558,
 0.010416666666666657]

In [33]:
# FAZ A ALGEBRA COM OS COEFICIENTES E RETORNA O POLINÔMIO APROXIMADOR FINAL
px = divs[0]
for i, coef in enumerate(divs[1:]):
    mult = 1
    for j in range(i+1):
        mult *= (X - x[j])
    px += coef*mult
px.simplify().subs(X, 1/3)
# sp.nsimplify(px.simplify().subs(X, 1/3), tolerance=0.001)

1.14660493827161

In [45]:
################################## CASO CONTÍNUO ###############################
# SE VOCÊ TIVER A FUNÇÃO A SER APROXIMADA, UTILIZAR ESSAS ULTIMAS CÉLULAS
# MUDAR f PARA A FUNÇÃO (USANDO SOMENTE SP. PARA QUE O MÓDULO NÃO DÊ ERRO.) EX: SP.COS(), SP.EXP, ETC.
# COLOCAR EM APROX O PONTO A SER APROXIMADO E MUDAR OS VALORES DE X (SE TIVER Y TAMBEM)
f = (1/3)**X
aprox = 1/10
x = np.array([-2, 0, 1, 3])
y = (1/3)**x
n = len(x)-1
f_der = sp.diff(f, X, n+1)
f_der
# RETORNA A DERIVADA (N+1) DE f

1.45672579400657*0.333333333333333**x

In [46]:
# COLOCAR O PONTO QUE VAI GERAR O MAJORANTE NO LUGAR DE 1.2 E RODAR
maj = f_der.subs(X, -2)
err = 1
for xi in x:
    err *= (X - xi)
err = sp.Abs(err) * sp.Abs(maj)/sp.factorial(n+1)
err.subs(X, aprox)
# RETORNA O ERRO APROXIMADO

0.299411777885626